IMPORTING THE DEPENDECIES

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Download NLTK stopwords if not already available

nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
# Printing the stopwords 
# the stopwords are going to be removed

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data Pre-Processing

In [4]:
# loading the dataset to a pandas dataframe

new_dataset = pd.read_csv('Datasets/train.csv')

In [5]:
# Numbers of rolls and columns

new_dataset.shape

(20800, 5)

In [6]:
# print the first 5 rows

new_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# counting the number of missing values in the dataset

new_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
#replacing the null values with empty string

news_dataset = new_dataset.fillna('')

In [9]:
# # Combine 'author' and 'title' into one column to form the content of the news

news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [10]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [11]:
#seperating the data and label

X = news_dataset.drop(columns='label',axis = 1)
Y = news_dataset['label']

In [12]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

# Stemming Procedure

This is a process of reducing a word to its Root word

In [13]:
# Initialize a stemmer (used to reduce words to their root form)

port_stem = PorterStemmer()

In [14]:
def stemming(content):
    """
        This function cleans and processes the text:
        - Removes special characters and numbers, keeping only letters.
        - Converts all words to lowercase.
        - Splits the text into individual words.
        - Removes common words (stopwords) that do not add meaning.
        - Converts words to their root form (stemming) to generalize them.
    """
    stemmed_content = re.sub('[^a-zA-z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return  stemmed_content

In [15]:
# Apply the text preprocessing function to the dataset

news_dataset['content'] = news_dataset['content'].apply(stemming)

In [16]:
news_dataset['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [17]:
#seperating the data annd label into a numpy array
# Extract features (X) and labels (Y)

X = news_dataset['content'].values # The processed text data
Y = news_dataset['label'].values  # The labels (1 for real news, 0 for fake news)

In [18]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [19]:
Y.shape

(20800,)

In [20]:
# Convert text data into numerical form using TF-IDF (gives importance to words in text)

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [21]:
print(X)

  (0, 268)	0.2701012497770876
  (0, 2485)	0.36765196867972083
  (0, 2962)	0.24684501285337127
  (0, 3603)	0.3598939188262558
  (0, 3795)	0.27053324808454915
  (0, 4979)	0.23331696690935097
  (0, 7012)	0.2187416908935914
  (0, 7698)	0.24785219520671598
  (0, 8636)	0.2921251408704368
  (0, 8915)	0.36359638063260746
  (0, 13480)	0.2565896679337956
  (0, 15697)	0.2848506356272864
  (1, 1498)	0.2939891562094648
  (1, 1895)	0.15521974226349364
  (1, 2224)	0.3827320386859759
  (1, 2816)	0.19094574062359204
  (1, 3571)	0.26373768806048464
  (1, 5510)	0.7143299355715573
  (1, 6823)	0.1904660198296849
  (1, 16812)	0.30071745655510157
  (2, 2946)	0.3179886800654691
  (2, 3106)	0.46097489583229645
  (2, 5396)	0.3866530551182615
  (2, 5975)	0.3474613386728292
  (2, 9625)	0.49351492943649944
  :	:
  (20797, 3646)	0.2115550061362374
  (20797, 7049)	0.21799048897828685
  (20797, 8370)	0.22322585870464115
  (20797, 8994)	0.36160868928090795
  (20797, 9523)	0.29542040034203126
  (20797, 9593)	0.17455348

# Splitting the dataset into training & test data

In [22]:
# Split the data into training and testing sets (80% training, 20% testing)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size= 0.2,stratify = Y, random_state = 2 )

# Training the Model Logistic Regression

In [23]:
# Train a machine learning model (Logistic Regression is simple yet effective)

model = LogisticRegression()
model.fit(X_train,Y_train)

LogisticRegression()

# 
Evaluation

# Accuracy Score

In [24]:
#accuracy score in the training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [25]:
print('Accuracy Score of the training data :',training_data_accuracy)

Accuracy Score of the training data : 0.9864783653846154


In [26]:
# accuracy score in the test data

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [27]:
print('Accuracy Score of the training data :',test_data_accuracy)

Accuracy Score of the training data : 0.979326923076923


# Making a Predictive System

In [28]:
X_new = X_test[6]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('The news is Real.')
else:
  print('The news is fake.')

[0]
The news is Real.


In [29]:
print(Y_test[0])

1


In [30]:
def predict_news(news_text):
    """
    This function takes a news title or short description as input and:
    - Cleans and processes the text like before.
    - Converts it into the numerical format the model understands.
    - Uses our trained model to predict if it's fake or real.
    - Returns a friendly result!
    """
    processed_text = stemming(news_text)
    vectorized_text = vectorizer.transform([processed_text])
    prediction = model.predict(vectorized_text)[0]
    return '🚨 FAKE NEWS 🚨' if prediction == 0 else '✅ REAL NEWS ✅'

# Let's test it with user input
user_input = input("Enter a news headline to check if it's FAKE or REAL: ")
result = predict_news(user_input)
print(f'Result: {result}')

Enter a news headline to check if it's FAKE or REAL:  House Dem Aide: We Didn’t Even See Comey’s


Result: ✅ REAL NEWS ✅
